# Tensorflow 2 implementation of open-neural-apc

Copyright (c) 2020, Nico Jahn <br>
All rights reserved.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# extension to run open-neural-apc in google colab and retrieving all neccessary files
import sys
IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    import os
    os.system("apt -qq update && apt -qq install git-lfs")
    os.system("git lfs install")
    os.system("git clone https://github.com/nicojahn/open-neural-apc.git")
    os.chdir("open-neural-apc")

In [3]:
%%bash
# install and update required packages
python3 -m pip install --upgrade pip -q
python3 -m pip install -r requirements.txt -q

In [4]:
from utils import load_config, allow_growth
allow_growth()
# read the config file
# it includes more or less all hyperparameter used in the model and preprocessing/training step
data_parameter, model_parameter, training_parameter = load_config(verbose=0)

In [5]:
# parsing command line arguments and overwriting config if those are supplied
from argument_parser import parse_arguments, overwrite_config
parsed_arguments, _ = parse_arguments()
overwrite_config(parsed_arguments, data_parameter, model_parameter, training_parameter)

In [6]:
''' Loading the training data '''
from data_loader import DataLoader
training_data = DataLoader(training_parameter, data_parameter["data"], "training")

if len(training_data)>0:
    ''' Creating a model and save the config'''
    from napc import NeuralAPC
    # create model from config
    napc = NeuralAPC(model_parameter, training_parameter)
    napc.compile()
    napc.save()

    # writing config file into model folder
    import json
    new_config = {"data_parameter": data_parameter, 
                  "model_parameter": model_parameter, 
                  "training_parameter": training_parameter}
    with open(napc.config_path, 'w+') as config:
        config.write(json.dumps(new_config, sort_keys=True, indent=2))
    print(f"Model folder created and config saved: {napc.config_path}")

    '''Data generator initialization'''
    from data_generator import DataGenerator
    # this is class which preprocesses the training data every epoch
    # it creates the necessary labels/bounds and augments the data
    training_generator = DataGenerator(training_data, training_parameter, training=True,
                                       return_sequences=model_parameter["return_sequences"])
    
    ''' Training procedure'''
    napc.fit(training_generator, epochs=training_parameter["epochs"],
                   initial_epoch=napc.epoch, max_queue_size=len(training_generator),
                   workers=4, use_multiprocessing=False, callbacks=napc.callbacks)

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: GeForce RTX 2080 Ti, compute capability 7.5
Model folder created and config saved: ./models/2020-12-05_19:35:15.362662/config.json
Epoch 1/1000
24/24 [==============================] - 8s 316ms/step - loss: 11.8274 - accuracy_v2: 0.1609 - mae: 1.9199
Epoch 2/1000
24/24 [==============================] - 7s 296ms/step - loss: 11.3487 - accuracy_v2: 0.1372 - mae: 1.9363
Epoch 3/1000
24/24 [==============================] - 7s 310ms/step - loss: 10.9660 - accuracy_v2: 0.1894 - mae: 1.8548
Epoch 4/1000
24/24 [==============================] - 7s 292ms/step - loss: 11.1179 - accuracy_v2: 0.1386 - mae: 1.8945
Epoch 5/1000
24/24 [==============================] - 8s 316ms/step - loss: 10.6010 - accuracy_v2: 0.1535 - mae: 1.8405
Epoch 6/1000
24/24 [==============================] - 7s 308ms/step - loss: 

24/24 [==============================] - 7s 306ms/step - loss: 7.1029 - accuracy_v2: 0.2642 - mae: 1.5172
Epoch 67/1000
24/24 [==============================] - 7s 305ms/step - loss: 6.4160 - accuracy_v2: 0.2587 - mae: 1.4724
Epoch 68/1000
24/24 [==============================] - 7s 294ms/step - loss: 6.1540 - accuracy_v2: 0.3011 - mae: 1.4148
Epoch 69/1000
24/24 [==============================] - 7s 292ms/step - loss: 6.1916 - accuracy_v2: 0.2997 - mae: 1.4238
Epoch 70/1000
24/24 [==============================] - 7s 294ms/step - loss: 6.2694 - accuracy_v2: 0.3099 - mae: 1.4233
Epoch 71/1000
24/24 [==============================] - 7s 276ms/step - loss: 6.5939 - accuracy_v2: 0.2755 - mae: 1.4884
Epoch 72/1000
24/24 [==============================] - 7s 305ms/step - loss: 6.2180 - accuracy_v2: 0.2934 - mae: 1.4322
Epoch 73/1000
24/24 [==============================] - 7s 296ms/step - loss: 5.8900 - accuracy_v2: 0.2879 - mae: 1.4047
Epoch 74/1000
24/24 [==============================] -

24/24 [==============================] - 7s 300ms/step - loss: 4.8980 - accuracy_v2: 0.3826 - mae: 1.2449
Epoch 135/1000
24/24 [==============================] - 7s 283ms/step - loss: 4.5730 - accuracy_v2: 0.4258 - mae: 1.2038
Epoch 136/1000
24/24 [==============================] - 6s 257ms/step - loss: 4.7111 - accuracy_v2: 0.3979 - mae: 1.2448
Epoch 137/1000
24/24 [==============================] - 8s 318ms/step - loss: 4.7687 - accuracy_v2: 0.3532 - mae: 1.2554
Epoch 138/1000
24/24 [==============================] - 7s 302ms/step - loss: 4.7122 - accuracy_v2: 0.4000 - mae: 1.2406
Epoch 139/1000
24/24 [==============================] - 6s 258ms/step - loss: 4.7176 - accuracy_v2: 0.4612 - mae: 1.1670
Epoch 140/1000
24/24 [==============================] - 7s 296ms/step - loss: 4.7674 - accuracy_v2: 0.4063 - mae: 1.2507
Epoch 141/1000
24/24 [==============================] - 7s 302ms/step - loss: 4.8876 - accuracy_v2: 0.4341 - mae: 1.2341
Epoch 142/1000
24/24 [=========================

24/24 [==============================] - 7s 285ms/step - loss: 4.6386 - accuracy_v2: 0.3803 - mae: 1.2369
Epoch 202/1000
24/24 [==============================] - 7s 279ms/step - loss: 4.1935 - accuracy_v2: 0.4140 - mae: 1.1174
Epoch 203/1000
24/24 [==============================] - 7s 303ms/step - loss: 3.7555 - accuracy_v2: 0.4528 - mae: 1.0661
Epoch 204/1000
24/24 [==============================] - 7s 294ms/step - loss: 3.2633 - accuracy_v2: 0.4644 - mae: 0.9752
Epoch 205/1000
24/24 [==============================] - 8s 313ms/step - loss: 2.6137 - accuracy_v2: 0.5412 - mae: 0.8636
Epoch 206/1000
24/24 [==============================] - 7s 280ms/step - loss: 2.7463 - accuracy_v2: 0.5355 - mae: 0.8734
Epoch 207/1000
24/24 [==============================] - 7s 305ms/step - loss: 2.0899 - accuracy_v2: 0.5362 - mae: 0.8147
Epoch 208/1000
24/24 [==============================] - 7s 279ms/step - loss: 2.1797 - accuracy_v2: 0.5917 - mae: 0.7615
Epoch 209/1000
24/24 [=========================

24/24 [==============================] - 7s 292ms/step - loss: 0.5757 - accuracy_v2: 0.7658 - mae: 0.4077
Epoch 269/1000
24/24 [==============================] - 7s 275ms/step - loss: 0.7644 - accuracy_v2: 0.7384 - mae: 0.4617
Epoch 270/1000
24/24 [==============================] - 7s 284ms/step - loss: 0.9356 - accuracy_v2: 0.7189 - mae: 0.5022
Epoch 271/1000
24/24 [==============================] - 7s 312ms/step - loss: 1.1057 - accuracy_v2: 0.6924 - mae: 0.5467
Epoch 272/1000
24/24 [==============================] - 6s 266ms/step - loss: 1.1780 - accuracy_v2: 0.6819 - mae: 0.5891
Epoch 273/1000
24/24 [==============================] - 6s 267ms/step - loss: 0.9904 - accuracy_v2: 0.6950 - mae: 0.5451
Epoch 274/1000
24/24 [==============================] - 7s 296ms/step - loss: 0.7415 - accuracy_v2: 0.7508 - mae: 0.4432
Epoch 275/1000
24/24 [==============================] - 7s 307ms/step - loss: 0.6482 - accuracy_v2: 0.7650 - mae: 0.4187
Epoch 276/1000
24/24 [=========================

...

24/24 [==============================] - 7s 291ms/step - loss: 0.5952 - accuracy_v2: 0.7706 - mae: 0.3974
Epoch 671/1000
24/24 [==============================] - 7s 304ms/step - loss: 0.5678 - accuracy_v2: 0.7818 - mae: 0.3943
Epoch 672/1000
24/24 [==============================] - 7s 284ms/step - loss: 0.8188 - accuracy_v2: 0.8353 - mae: 0.3672
Epoch 673/1000
24/24 [==============================] - 7s 284ms/step - loss: 0.8268 - accuracy_v2: 0.7622 - mae: 0.4454
Epoch 674/1000
24/24 [==============================] - 7s 301ms/step - loss: 0.4267 - accuracy_v2: 0.8200 - mae: 0.3198
Epoch 675/1000
24/24 [==============================] - 7s 302ms/step - loss: 0.2053 - accuracy_v2: 0.8663 - mae: 0.2369
Epoch 676/1000
24/24 [==============================] - 7s 298ms/step - loss: 0.1717 - accuracy_v2: 0.8816 - mae: 0.2221
Epoch 677/1000
24/24 [==============================] - 7s 297ms/step - loss: 0.1268 - accuracy_v2: 0.8909 - mae: 0.1964
Epoch 678/1000
24/24 [=========================

24/24 [==============================] - 7s 273ms/step - loss: 0.0678 - accuracy_v2: 0.9309 - mae: 0.1450
Epoch 738/1000
24/24 [==============================] - 7s 288ms/step - loss: 0.0770 - accuracy_v2: 0.9255 - mae: 0.1526
Epoch 739/1000
24/24 [==============================] - 6s 268ms/step - loss: 0.0728 - accuracy_v2: 0.9265 - mae: 0.1486
Epoch 740/1000
24/24 [==============================] - 7s 307ms/step - loss: 0.0810 - accuracy_v2: 0.9217 - mae: 0.1598
Epoch 741/1000
24/24 [==============================] - 7s 284ms/step - loss: 0.0735 - accuracy_v2: 0.9283 - mae: 0.1507
Epoch 742/1000
24/24 [==============================] - 7s 296ms/step - loss: 0.0811 - accuracy_v2: 0.9264 - mae: 0.1565
Epoch 743/1000
24/24 [==============================] - 6s 263ms/step - loss: 0.0813 - accuracy_v2: 0.9215 - mae: 0.1554
Epoch 744/1000
24/24 [==============================] - 7s 308ms/step - loss: 0.0788 - accuracy_v2: 0.9236 - mae: 0.1535
Epoch 745/1000
24/24 [=========================

24/24 [==============================] - 7s 281ms/step - loss: 0.5771 - accuracy_v2: 0.7893 - mae: 0.3875
Epoch 805/1000
24/24 [==============================] - 6s 241ms/step - loss: 0.3591 - accuracy_v2: 0.8470 - mae: 0.3009
Epoch 806/1000
24/24 [==============================] - 7s 300ms/step - loss: 0.4112 - accuracy_v2: 0.8320 - mae: 0.3144
Epoch 807/1000
24/24 [==============================] - 7s 305ms/step - loss: 0.2814 - accuracy_v2: 0.8722 - mae: 0.2524
Epoch 808/1000
24/24 [==============================] - 7s 300ms/step - loss: 0.2118 - accuracy_v2: 0.8873 - mae: 0.2156
Epoch 809/1000
24/24 [==============================] - 7s 304ms/step - loss: 0.1718 - accuracy_v2: 0.8965 - mae: 0.2042
Epoch 810/1000
24/24 [==============================] - 7s 275ms/step - loss: 0.5588 - accuracy_v2: 0.8441 - mae: 0.3047
Epoch 811/1000
24/24 [==============================] - 7s 276ms/step - loss: 0.3578 - accuracy_v2: 0.8254 - mae: 0.3149
Epoch 812/1000
24/24 [=========================

24/24 [==============================] - 7s 306ms/step - loss: 0.0524 - accuracy_v2: 0.9473 - mae: 0.1274
Epoch 872/1000
24/24 [==============================] - 6s 271ms/step - loss: 0.0444 - accuracy_v2: 0.9554 - mae: 0.1187
Epoch 873/1000
24/24 [==============================] - 7s 297ms/step - loss: 0.0500 - accuracy_v2: 0.9491 - mae: 0.1215
Epoch 874/1000
24/24 [==============================] - 7s 289ms/step - loss: 0.0501 - accuracy_v2: 0.9538 - mae: 0.1226
Epoch 875/1000
24/24 [==============================] - 7s 277ms/step - loss: 0.0473 - accuracy_v2: 0.9519 - mae: 0.1227
Epoch 876/1000
24/24 [==============================] - 7s 292ms/step - loss: 0.0454 - accuracy_v2: 0.9517 - mae: 0.1192
Epoch 877/1000
24/24 [==============================] - 7s 277ms/step - loss: 0.0631 - accuracy_v2: 0.9355 - mae: 0.1341
Epoch 878/1000
24/24 [==============================] - 7s 296ms/step - loss: 0.0586 - accuracy_v2: 0.9384 - mae: 0.1301
Epoch 879/1000
24/24 [=========================

24/24 [==============================] - 7s 281ms/step - loss: 0.0474 - accuracy_v2: 0.9579 - mae: 0.1205
Epoch 939/1000
24/24 [==============================] - 7s 311ms/step - loss: 0.0510 - accuracy_v2: 0.9520 - mae: 0.1260
Epoch 940/1000
24/24 [==============================] - 7s 300ms/step - loss: 0.0462 - accuracy_v2: 0.9572 - mae: 0.1244
Epoch 941/1000
24/24 [==============================] - 7s 290ms/step - loss: 0.0453 - accuracy_v2: 0.9567 - mae: 0.1169
Epoch 942/1000
24/24 [==============================] - 7s 287ms/step - loss: 0.0427 - accuracy_v2: 0.9566 - mae: 0.1157
Epoch 943/1000
24/24 [==============================] - 7s 302ms/step - loss: 0.0392 - accuracy_v2: 0.9607 - mae: 0.1109
Epoch 944/1000
24/24 [==============================] - 7s 299ms/step - loss: 0.0683 - accuracy_v2: 0.9450 - mae: 0.1304
Epoch 945/1000
24/24 [==============================] - 8s 317ms/step - loss: 0.0882 - accuracy_v2: 0.9450 - mae: 0.1368
Epoch 946/1000
24/24 [=========================

In [7]:
''' For all of you which don't have the training data and just want to execute the notebooks '''
if len(training_data) == 0:
    data_parameter, model_parameter, training_parameter = load_config("models/config.json",
                                                                     verbose=False)
    from napc import NeuralAPC
    napc = NeuralAPC(model_parameter, training_parameter)
    # Loading the included model (it has no subdirectory)
    napc.load_model(epoch=10000, model_path="models/")
    # The model_path of the model is not 'models/', but the previously created subdirectory
    # You could now train it further/save it/ etc.
    napc.compile()

In [8]:
# set last lstm layer from last output to all outputs
import tensorflow as tf
model = napc._model
model.get_layer(f"CoreLayer{model_parameter['lstm_depth']-1}").return_sequences = True

''' Produce videos on all validation sequences or just validate the model '''
# de-/activate video creation
create_vids = False
if create_vids:
    ! apt -qq update && apt -qq install -y ffmpeg

# copy dict from training and modify the concatenation
validation_parameter = training_parameter.copy()
validation_parameter["concatenation_length"] = 1

# read the validation data
from data_loader import DataLoader
validation_data = DataLoader(validation_parameter, data_parameter["data"], "validation")

# process them (i need the bounds in y for the accuracy and the videos)
from data_generator import DataGenerator
validation_generator = DataGenerator(validation_data, validation_parameter, training=False,
                                     return_sequences=True)

from tqdm.notebook import trange
accuracy = []
for batch_idx in trange(len(validation_generator), desc="Batches done", unit="batches"):
    # get batch, predict and calculate accuracy
    x,y = validation_generator[batch_idx]
    predictions = model(x)
    accuracy += [napc.accuracy(y, predictions)]
    # creates my videos
    if create_vids:
        from video_generator import create_videos
        create_videos(x, y, predictions, napc.epoch, batch_idx*validation_parameter["batch_size"], 
                     data_parameter["class_names"])

In [9]:
# Since I'm not allowed to upload more sequences and I don't want to publish a perfect model
# (therefore, I haven't tested this one) the accuracy is just an approximation of the true capabilities
# The 'validation' data in this case is in fact a test set (last epoch was chosen without selection)
# In practice someone would use k-fold-CV and would reason about the average performance
import numpy as np
# So let's have a look how well the model does...
print(f"Accuracy: {100*np.mean(accuracy)} %")

Accuracy: 79.98046875 %
